In [ ]:
import sys
import os
sys.path.insert(0, os.path.abspath('..'))

import pytest
import time
from core.common.performance_monitor import MetricSnapshot, PerformanceHistory, ResourceMonitor

## Test MetricSnapshot

In [ ]:
def test_metric_snapshot_creation():
    """Test MetricSnapshot creation"""
    snapshot = MetricSnapshot(
        timestamp=time.time(),
        cpu_percent=50.0,
        memory_percent=60.0,
        disk_percent=70.0,
        network_sent=1000,
        network_recv=2000
    )
    
    assert snapshot.cpu_percent == 50.0
    assert snapshot.memory_percent == 60.0
    assert snapshot.disk_percent == 70.0
    assert snapshot.network_sent == 1000
    assert snapshot.network_recv == 2000
    print("✅ MetricSnapshot creation test passed")

test_metric_snapshot_creation()

## Test PerformanceHistory

In [ ]:
def test_performance_history_init():
    """Test PerformanceHistory initialization"""
    history = PerformanceHistory(max_samples=10)
    
    assert history.max_samples == 10
    assert len(history.history) == 0
    print("✅ PerformanceHistory init test passed")

test_performance_history_init()

In [ ]:
def test_add_snapshot():
    """Test adding snapshots to history"""
    history = PerformanceHistory(max_samples=5)
    
    # Add 3 snapshots
    for i in range(3):
        snapshot = MetricSnapshot(
            timestamp=time.time(),
            cpu_percent=50.0 + i,
            memory_percent=60.0,
            disk_percent=70.0
        )
        history.add_snapshot(snapshot)
    
    assert len(history.history) == 3
    print("✅ Add snapshot test passed")

test_add_snapshot()

In [ ]:
def test_max_samples_limit():
    """Test that history respects max_samples limit"""
    history = PerformanceHistory(max_samples=3)
    
    # Add 5 snapshots (exceeds max)
    for i in range(5):
        snapshot = MetricSnapshot(
            timestamp=time.time(),
            cpu_percent=50.0 + i,
            memory_percent=60.0,
            disk_percent=70.0
        )
        history.add_snapshot(snapshot)
    
    assert len(history.history) == 3
    assert history.history[0].cpu_percent == 52.0  # First should be removed
    print("✅ Max samples limit test passed")

test_max_samples_limit()

In [ ]:
def test_get_average():
    """Test calculating average metrics"""
    history = PerformanceHistory(max_samples=10)
    
    # Add snapshots with known values
    for i in range(3):
        snapshot = MetricSnapshot(
            timestamp=time.time(),
            cpu_percent=30.0 + (i * 10),  # 30, 40, 50
            memory_percent=60.0,
            disk_percent=70.0
        )
        history.add_snapshot(snapshot)
    
    avg = history.get_average()
    assert avg['cpu'] == 40.0  # (30+40+50)/3 = 40
    assert avg['memory'] == 60.0
    assert avg['disk'] == 70.0
    print("✅ Get average test passed")

test_get_average()

In [ ]:
def test_get_peak():
    """Test finding peak metrics"""
    history = PerformanceHistory(max_samples=10)
    
    # Add snapshots with varying values
    for cpu in [30.0, 80.0, 50.0]:
        snapshot = MetricSnapshot(
            timestamp=time.time(),
            cpu_percent=cpu,
            memory_percent=60.0,
            disk_percent=70.0
        )
        history.add_snapshot(snapshot)
    
    peak = history.get_peak()
    assert peak['cpu'] == 80.0
    assert peak['memory'] == 60.0
    assert peak['disk'] == 70.0
    print("✅ Get peak test passed")

test_get_peak()

In [ ]:
def test_get_trend():
    """Test trend detection"""
    history = PerformanceHistory(max_samples=10)
    
    # Add increasing CPU values
    for i in range(5):
        snapshot = MetricSnapshot(
            timestamp=time.time(),
            cpu_percent=20.0 + (i * 10),  # 20, 30, 40, 50, 60
            memory_percent=50.0,
            disk_percent=70.0
        )
        history.add_snapshot(snapshot)
    
    trend = history.get_trend()
    assert trend['cpu'] == 'increasing'
    assert trend['memory'] == 'stable'
    print(f"✅ Get trend test passed: {trend}")

test_get_trend()

## Test ResourceMonitor

In [ ]:
def test_resource_monitor_init():
    """Test ResourceMonitor initialization"""
    monitor = ResourceMonitor(
        cpu_threshold=80.0,
        memory_threshold=85.0,
        disk_threshold=90.0
    )
    
    assert monitor.cpu_threshold == 80.0
    assert monitor.memory_threshold == 85.0
    assert monitor.disk_threshold == 90.0
    assert hasattr(monitor, 'history')
    print("✅ ResourceMonitor init test passed")

test_resource_monitor_init()

In [ ]:
def test_resource_monitor_update():
    """Test ResourceMonitor update method"""
    monitor = ResourceMonitor()
    
    monitor.update(cpu=50.0, memory=60.0, disk=70.0)
    
    assert len(monitor.history.history) == 1
    assert monitor.history.history[0].cpu_percent == 50.0
    print("✅ ResourceMonitor update test passed")

test_resource_monitor_update()

In [ ]:
def test_health_score_calculation():
    """Test health score calculation"""
    monitor = ResourceMonitor()
    
    # Perfect system (0% usage)
    monitor.update(cpu=0.0, memory=0.0, disk=0.0)
    health = monitor.get_health_score()
    assert health == 100
    
    # High usage system
    monitor.update(cpu=90.0, memory=85.0, disk=80.0)
    health = monitor.get_health_score()
    assert health < 20  # Should be low
    print(f"✅ Health score test passed: {health}")

test_health_score_calculation()

In [ ]:
def test_alert_callback():
    """Test alert callback mechanism"""
    monitor = ResourceMonitor(cpu_threshold=50.0)
    alerts_received = []
    
    def alert_handler(metric, value, threshold):
        alerts_received.append((metric, value, threshold))
    
    monitor.add_alert_callback(alert_handler)
    
    # Trigger CPU alert
    monitor.update(cpu=80.0, memory=40.0, disk=30.0)
    
    assert len(alerts_received) == 1
    assert alerts_received[0][0] == 'cpu'
    assert alerts_received[0][1] == 80.0
    print(f"✅ Alert callback test passed: {alerts_received}")

test_alert_callback()

## Run All Tests

In [ ]:
# Run all tests
pytest.main(['-v', '--tb=short', __file__])